# COGS 108 - Data Checkpoint

# Names

- Anna Wang
- Chloe Salem
- Kristy Liou
- Maxtierney Arias
- Zeven Vidmar Barker

<a id='research_question'></a>
# Research Question

Can we predict which state in the USA will be covid-free first based on current hospital records, state regulations, and population?

# Dataset(s)

*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name: **Population, Population Changes, and Estimates**
- Link to the dataset: https://www2.census.gov/programs-surveys/popest/datasets/2010-2020/national/totals/nst-est2020.csv
- Number of observations: The US Census dataset from 2010 includes populations of the country, states, and regions as well as estimates of each for every year leading up to 2020. This dataset is the best available given that the 2020 census is still being processed.


- Dataset Name: **US State Vaccinations**
- Link to the dataset: https://github.com/owid/covid-19-data/blob/master/public/data/vaccinations/us_state_vaccinations.csv
- It has 2104 observations, which include data of information of how many people from each state has been vaccinated starting from January 12, 2021.


- Dataset Name: **COVID Tracking**
- Link to the dataset: https://covidtracking.com/data
- It has 2,006 observations, that shows us the number of covid cases and death for each state in the US since Janurary 12, 2020.
1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

# Setup

In [51]:
## YOUR CODE HERE67tyt7
import pandas as pd
import seaborn as sns
import numpy as np

population = pd.read_csv("https://www2.census.gov/programs-surveys/popest/datasets/2010-2020/national/totals/nst-est2020.csv")
vaccinations = pd.read_csv('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/us_state_vaccinations.csv')
#includes hospitalization and covid cases over time
case_tracking = pd.read_csv('https://covidtracking.com/data/download/all-states-history.csv')

# Data Cleaning

The three datasets we used were already in a tidy format, with variables representing every distinct measurement made by the sources in the columns, and separate observations in the rows. Since we did remove observations unrelated to states in the Census, and we just wanted the states and their 2020 population estimates, we reset the index for `population`. For the other datasets, we just removed the variables that were unnecessary for our analysis.

In [52]:
# Removing the regions population
population = population[5::]
# Remove columns with years outside 2020
population = population.drop(population.columns[7:-1], 1)
# Remove unnecessary variables related to region and identifiers
population = population.drop(['STATE','SUMLEV', 'DIVISION', 'REGION','CENSUS2010POP','ESTIMATESBASE2010'], 1)
# Set index to be the name of the state, rather than an arbitrary number
population.set_index(['NAME'], inplace=True)
population.head()

,POPESTIMATE2020
NAME,
Alabama,4921532
Alaska,731158
Arizona,7421401
Arkansas,3030522
California,39368078
Colorado,5807719
Connecticut,3557006
Delaware,986809
District of Columbia,712816


In [55]:
#drop columns unrelated to research question
vaccinations = vaccinations.drop(['daily_vaccinations_per_million', 'share_doses_used', 'daily_vaccinations_per_million', 'share_doses_used'], 1)
vaccinations.head()

,date,location,total_vaccinations,total_distributed,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations_raw,daily_vaccinations
0,2021-01-12,Alabama,78134.0,377025.0,70861.0,0.15,1.59,7270.0,1.44,7.69,NaN,NaN
1,2021-01-13,Alabama,84040.0,378975.0,74792.0,0.19,1.71,9245.0,1.52,7.73,5906.0,5906.0
2,2021-01-14,Alabama,92300.0,435350.0,80480.0,NaN,1.88,NaN,1.64,8.88,8260.0,7083.0
3,2021-01-15,Alabama,100567.0,444650.0,86956.0,0.27,2.05,13488.0,1.77,9.07,8267.0,7478.0
4,2021-01-16,Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7557.0,7498.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2098,2021-02-08,Wyoming,76457.0,101650.0,58775.0,2.89,13.21,16729.0,10.15,17.56,4599.0,2534.0
2099,2021-02-09,Wyoming,76457.0,113175.0,58775.0,2.89,13.21,16729.0,10.15,19.56,0.0,2491.0
2100,2021-02-10,Wyoming,79515.0,117475.0,61170.0,3.14,13.74,18180.0,10.57,20.30,3058.0,2677.0
2101,2021-02-11,Wyoming,79515.0,118875.0,61170.0,3.14,13.74,18180.0,10.57,20.54,0.0,2677.0


In [56]:
#dropping more unrelated columns to research question - including deaths, antibody tests, negative results, and positive results related to type of test
case_tracking = case_tracking.drop(['dataQualityGrade','deathProbable', 'death', 'deathConfirmed', 'deathIncrease', 'negative',
       'negativeIncrease', 'negativeTestsAntibody',
       'negativeTestsPeopleAntibody', 'negativeTestsViral', 'totalTestsAntibody', 'totalTestsAntigen',
       'totalTestsPeopleAntibody', 'totalTestsPeopleAntigen',
       'totalTestsPeopleViral', 'totalTestsPeopleViralIncrease',
       'totalTestsViral', 'totalTestsViralIncrease','positiveCasesViral', 'positiveIncrease', 'positiveScore',
       'positiveTestsAntibody', 'positiveTestsAntigen',
       'positiveTestsPeopleAntibody', 'positiveTestsPeopleAntigen',
       'positiveTestsViral', 'inIcuCumulative','inIcuCurrently','onVentilatorCumulative', 'onVentilatorCurrently'], 1)
case_tracking.head()

KeyError: "['dataQualityGrade' 'deathProbable' 'death' 'deathConfirmed'\n 'deathIncrease' 'negative' 'negativeIncrease' 'negativeTestsAntibody'\n 'negativeTestsPeopleAntibody' 'negativeTestsViral' 'totalTestsAntibody'\n 'totalTestsAntigen' 'totalTestsPeopleAntibody' 'totalTestsPeopleAntigen'\n 'totalTestsPeopleViral' 'totalTestsPeopleViralIncrease' 'totalTestsViral'\n 'totalTestsViralIncrease' 'positiveCasesViral' 'positiveIncrease'\n 'positiveScore' 'positiveTestsAntibody' 'positiveTestsAntigen'\n 'positiveTestsPeopleAntibody' 'positiveTestsPeopleAntigen'\n 'positiveTestsViral' 'inIcuCumulative' 'inIcuCurrently'\n 'onVentilatorCumulative' 'onVentilatorCurrently'] not found in axis"

# Project Proposal (updated)

| Meeting Date  | Meeting Time| Completed Before Meeting  | Discuss at Meeting |
|---|---|---|---|
| 1/20  |  1 PM | Read & Think about COGS 108 expectations  | Determine best form of communication; Discuss and decide on final project topic; discuss hypothesis; begin background research | 
| 1/26  |  10 AM |  Do background research on topic | Discuss ideal dataset(s) and ethics; draft project proposal | 
| 2/1  | 10 AM  | Edit, finalize, and submit proposal; Search for datasets  | Discuss Wrangling and possible analytical approaches; Assign group members to lead each specific part   |
| 2/13  | 4 PM  | Import & Wrangle Data; EDA | Review/Edit wrangling/EDA; Discuss Analysis Plan   |
| 2/23  | 12 PM  | Finalize wrangling/EDA; Begin Analysis| Discuss/edit Analysis; Complete project check-in |
| 3/13  | 12 PM  | Complete analysis; Draft results/conclusion/discussion| Discuss/edit full project |
| 3/19  | Before 11:59 PM  | NA | Turn in Final Project & Group Project Surveys |